In [346]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [347]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df.fillna(0)

df.sample(20)



,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
217295,0x6d183ac3db4e904f5cb34b09d72e78b5fc5c69c5,in,OTHER,OTHER,OTHER,1.277185e+02,2,2,Ethereum
178615,0x740b45e662435b817719ed252f241409b116d98c,out,OTHER,OTHER,OTHER,4.205310e+03,2,2,Ethereum
7955,0x4ab66d064e65821dd2f8bfe5e0c4f3024fc585a1,in,oneinch,AggregationRouterV5,swap,7.031090e+05,1,1,Ethereum
61777,0x669a022aaeb05a7feaa7b0619240e26296a1b88f,out,0xc5068b55b98e8433622bd3845e2579026dea1e92,0,transfer,9.203924e+04,2,2,Ethereum
52949,0xf3fa1a1a0cf4e71181adfa4301d612c8e0a7ba7a,out,0xac9b0b0ae39c9c1fe3962090e971257ffd60b29c,0,transfer,1.309207e+05,1,1,Ethereum
167233,0x48c04ed5691981c42154c6167398f95e8f38a7ff,in,0x258a996d3930dd4f95cd25f138e1dfdcb06f5547,0,transfer,1.108193e+04,1,1,Ethereum
164100,0xcb0bed06e1ea1c4397a3a53ec1f2db562bd4f196,in,0xa6f5bdf94b2d3050ae28a9b430a7ecf3f1bb0dd2,0,transfer,2.613277e+04,1,1,Ethereum
204150,0x8d28a0d98f2cc02fc28261f31f3aa9ea4ac3f8d3,in,0x02d2eddc7b8c99b1cbf5506347ff1e3585396b98,0,transfer,1.008522e+05,2,2,Ethereum
91464,0xf35afe47d3ae8fca8fa923ef9514430016cba3e5,out,0x4d54701f5990017a61e313746c58a60cdce160fd,0,transfer,1.208499e+05,1,1,Ethereum
110552,0x2dbf483371c20efb7dd47c3daab1ef3d5753ae97,out,0x4bf1ae0b233e0fdc1f99173e4d917c86afea215b,0,transfer,1.637012e+04,1,1,Ethereum


In [348]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [349]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/4210915753.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [350]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net'\
                ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share'
                ,'address' \
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [351]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [352]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})

df_action_by_app = df_addr.groupby(['chain','project','address']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
})

df_action_by_app.reset_index(inplace=True)

for val in val_cols:
        df_action_by_app[val + '_pct_share'] = df_action_by_app.groupby(['chain','address'])[val].transform(lambda x: x / x.sum())

df_action_by_app = df_action_by_app.groupby(['chain','project']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        # 'usd_value':'mean'
        # ,'usd_value_in':'mean'
        # ,'usd_value_out':'mean'
        # ,'usd_value_net':'mean'
        # ,'address':'nunique'
})


df_action.reset_index(inplace=True)
df_action_by_app.reset_index(inplace=True)

df_action.name = 'df_action'
df_action_by_app.name = 'df_action_by_app'

display(df_action)

,chain,project,contract,method,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,usd_value,usd_value_in,usd_value_out,usd_value_net,address
0,Arbitrum,0x0000000000be6d8381e3a01c19e3b3c2b6d9c7cd,0,0xed5b0df3,0.232680,0.063109,0.365967,0.063109,5.829281e+06,695814.462200,5.133467e+06,695814.462200,1
1,Arbitrum,0x000000d4728ded3cc178ffa58355e6b57b931ee2,0,0xed5b0df3,0.367702,0.028867,0.634033,0.028867,9.211949e+06,318280.444237,8.893669e+06,318280.444237,1
2,Arbitrum,0x000c3ff63086a0a34b3a4750a66adf206753fc6b,0,transfer,0.000233,0.000456,0.000000,0.000456,1.007082e+04,10070.820000,0.000000e+00,10070.820000,1
3,Arbitrum,0x00680642adb56ea659d96a91e81099c43886f513,0,transfer,0.575438,0.701914,0.297549,0.701914,4.997175e+05,418897.352500,8.082018e+04,418897.352500,1
4,Arbitrum,0x008ba48b1c005edbc76d75c7143c868576f1fbbf,0,transfer,0.000262,0.000000,0.000536,0.000000,1.129990e+04,0.000000,1.129990e+04,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139229,Optimism,Wepiggy,PEther,borrow,0.001958,0.004165,0.000000,0.004165,2.318055e+05,231805.500000,0.000000e+00,231805.500000,1
139230,Optimism,Wepiggy,PEther,repayBorrow,0.000144,0.000000,0.000271,0.000000,1.699907e+04,0.000000,1.699907e+04,0.000000,1
139231,Optimism,Woofi,0,withdraw,0.030797,0.060700,0.000000,0.060700,1.004902e+05,100490.201009,0.000000e+00,100490.201009,1
139232,Optimism,Woofi,WooSuperChargerVault,deposit,0.030780,0.000000,0.062481,0.000000,1.004344e+05,0.000000,1.004344e+05,0.000000,1


In [353]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        # d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

,index,chain,project,contract,method,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,usd_value,usd_value_in,usd_value_out,usd_value_net,address,action
124011,124011,Ethereum,0xe796e0360e557963885be268382fff54ec9f2d12,0,transfer,0.000413,0.000000,0.000824,0.000000,201416.400000,0.000000,201416.400000,0.000000,1,0xe796e0360e557963885be268382fff54ec9f2d12 - 0...
64340,64340,Ethereum,0x754249499bdc5e46a339ac36fef816a48235fbda,0,0x,0.000542,0.001123,0.000000,0.001123,46309.372847,46309.372847,0.000000,46309.372847,1,0x754249499bdc5e46a339ac36fef816a48235fbda - 0...
11338,11338,Ethereum,0x101107788a3fac7a4806dab428f64d5692642e82,0,0xafef944f,0.000059,0.000108,0.000000,0.000108,21913.886952,21913.886952,0.000000,21913.886952,1,0x101107788a3fac7a4806dab428f64d5692642e82 - 0...
131244,131244,Ethereum,0xf56ee76c3d3dda04c50aba3bc80f8f66531b4d9e,0,0x,0.035525,0.045475,0.000000,0.045475,90671.716201,90671.716201,0.000000,90671.716201,1,0xf56ee76c3d3dda04c50aba3bc80f8f66531b4d9e - 0...
59327,59327,Ethereum,0x6b934737114ca442a25e4397a57d074e010de67a,0,transfer,0.000252,0.000000,0.000507,0.000000,44118.892777,0.000000,44118.892777,0.000000,1,0x6b934737114ca442a25e4397a57d074e010de67a - 0...
16217,16217,Ethereum,0x195d509a0781ac1d92e0e5644e3fa87a5c1fa00f,0,transfer,0.000046,0.000092,0.000000,0.000092,15103.611587,15103.611587,0.000000,15103.611587,1,0x195d509a0781ac1d92e0e5644e3fa87a5c1fa00f - 0...
40805,40805,Ethereum,0x486dbf19cd296c7e4b61b82dd4a1fb40df49a5df,0,transfer,0.072541,0.155679,0.027039,0.155679,495897.996889,245506.852793,250391.144096,245506.852793,5,0x486dbf19cd296c7e4b61b82dd4a1fb40df49a5df - 0...
47565,47565,Ethereum,0x555bc35e20060a6b52690130e63e62d75ca5a824,0,transfer,0.000009,0.000009,0.000008,0.000009,20028.443983,10014.221992,10014.221992,10014.221992,1,0x555bc35e20060a6b52690130e63e62d75ca5a824 - 0...
18066,18066,Ethereum,0x1cfd8781a9357d93a9651d8efe1ff8cf2654656f,0,transferFrom,0.000137,0.000233,0.000000,0.000233,11782.859400,11782.859400,0.000000,11782.859400,1,0x1cfd8781a9357d93a9651d8efe1ff8cf2654656f - 0...
82307,82307,Ethereum,0x97fe535ceed71481d6cd6f2ca3b54e3c1b9cbae5,0,transfer,0.021171,0.000000,0.042342,0.000000,51255.552823,0.000000,51255.552823,0.000000,1,0x97fe535ceed71481d6cd6f2ca3b54e3c1b9cbae5 - 0...


In [354]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )

        print(sumdf[['usd_value_out_pct_share_100']].sum())

Dollar-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1668449685.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
2562,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,9.523727,9.255335,4
2561,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,5.702290,4.862586,1
1927,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,5.504098,0.000000,14
2563,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,4.944629,5.567669,1
2015,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,3.998533,0.000000,36
536,Arbitrum,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3 - 0...,3.286356,1.529717,78
343,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,3.059880,0.000000,65
2568,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,2.852370,3.406012,131
2020,Arbitrum,gmx,gmx - Vault | 0x364e2311,2.278467,0.000000,58
1891,Arbitrum,OTHER,OTHER - OTHER | OTHER,2.271972,2.345757,321


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1668449685.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
663,Optimism,Celer,Celer - Bridge | addLiquidity,17.775668,0.000000,16
944,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,15.668466,15.976778,1
761,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,5.628581,0.000000,51
950,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,3.548631,3.762533,2
736,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,2.776817,4.930665,16
633,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,2.594831,0.006871,59
375,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,0xb1feccccaad6429899a9401dcb19f58a19d1afe6 - 0...,2.323820,0.000000,1
666,Optimism,Celer,Celer - Bridge | send,1.833812,0.000000,25
741,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,1.822989,0.000000,11
804,Optimism,Socket,Socket - 0 | outboundTransferTo,1.733445,0.000000,18


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1668449685.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
576,Optimism,Uniswap,23.300819,25.730368,119
533,Optimism,Celer,20.354813,24.457077,93
553,Optimism,Multichain,6.270284,6.020605,132
549,Optimism,Kyberswap,5.180304,5.265309,56
524,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,2.639540,0.006871,62
526,Optimism,Aave,2.598473,4.172976,114
577,Optimism,Uniswap V3,2.470609,3.366635,48
571,Optimism,Sonne Finance,2.432443,2.811540,94
357,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,2.323820,0.000000,1
537,Optimism,Dforce,1.920007,1.210225,60


usd_value_out_pct_share_100    100.0
dtype: float64


In [355]:
print('Address-Weighted')
# print(df_action.columns)
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # display(tdf)
        tdf = tdf[tdf['address'] >=20]
        sumdf = tdf
        print('Sum by Action')
        # sumdf = tdf.groupby(['chain','project','action']).sum()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        # print(sumdf.columns)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = df_action_by_app.copy()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )
        # print(sumdf[['usd_value_out_pct_share_100']].sum()) #SHould be > 100 since it's avg by address

Address-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/664403606.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
17,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,52.366119,0.000000,36.0
19,Arbitrum,gmx,gmx - Vault | 0x364e2311,51.861224,0.000000,58.0
9,Arbitrum,0xf89d7b9c864f589bbf53a82105107622b35eaa40,0xf89d7b9c864f589bbf53a82105107622b35eaa40 - 0...,46.153846,17.144626,26.0
13,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,42.060198,0.000000,27.0
25,Arbitrum,sushi,sushi - Pair | swapExactTokensForETH,38.122838,0.000000,20.0
7,Arbitrum,0x892785f33cdee22a30aef750f285e18c18040c3e,0x892785f33cdee22a30aef750f285e18c18040c3e - 0...,35.303184,0.000000,30.0
8,Arbitrum,0xb38e8c17e38363af6ebdcb3dae12e0243582891d,0xb38e8c17e38363af6ebdcb3dae12e0243582891d - 0...,34.375000,18.606047,33.0
23,Arbitrum,gmx,gmx - sGMX | 0xf3daeacc,32.593963,0.000000,61.0
28,Arbitrum,uniswap_v3,uniswap_v3 - Pair | mint,28.874147,0.000000,27.0
3,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,28.503866,0.000000,65.0


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/664403606.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).mean()


...

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/664403606.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
10,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,33.031674,0.000000,51.0
14,Optimism,Velodrome,Velodrome - Pair | addLiquidity,30.905623,0.000000,21.0
8,Optimism,Celer,Celer - Bridge | send,25.135731,0.000000,25.0
1,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,21.100266,0.117743,59.0
4,Optimism,Aave,Aave - AToken | repay,20.145970,0.000000,23.0
5,Optimism,Aave,Aave - AToken | supply,13.831759,0.000000,23.0
13,Optimism,Uniswap,Uniswap - Pool | multicall,12.931811,11.305158,21.0
11,Optimism,OTHER,OTHER - OTHER | OTHER,3.782918,10.768166,188.0
2,Optimism,1inch,1inch - AggregationRouterV5 | swap,2.878059,23.089112,57.0
0,Optimism,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0xacd03d601e5bb1b275bb94076ff46ed9d753435a - 0...,0.000000,19.371645,23.0


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/664403606.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).mean()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
9,Optimism,Velodrome,30.905623,0.000000,21.0
1,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,21.100266,0.117743,59.0
5,Optimism,Multichain,16.515837,17.171564,50.0
8,Optimism,Uniswap,12.931811,11.305158,21.0
4,Optimism,Celer,12.567866,10.473531,28.0
3,Optimism,Aave,8.494432,12.073350,23.5
6,Optimism,OTHER,3.782918,10.768166,188.0
2,Optimism,1inch,2.878059,23.089112,57.0
0,Optimism,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0.000000,19.371645,23.0
7,Optimism,Stargate Finance,0.000000,35.821886,21.0
